In [1]:
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import numpy as np
import math  
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
sns.set(style="darkgrid")
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%config ZMQInteractiveShell.ast_node_interactivity='all'
%pprint

Pretty printing has been turned OFF


## 数据地址

In [2]:
train_path = '../data/underexpose_train'  
test_path = '../data/underexpose_test'
feat_path='../user_data/model_data'
train_path_click=train_path+'/underexpose_train_click-{}.csv'
test_path_click=test_path+'/underexpose_test_click-{0}/underexpose_test_click-{0}.csv'
test_path_click_qtime=test_path+'/underexpose_test_click-{0}/underexpose_test_qtime-{0}.csv'
train_path_item=train_path+'/underexpose_item_feat.csv'
train_path_user=train_path+'/underexpose_user_feat.csv'

## 存放特征的地址

In [3]:
item_feat_path=feat_path+'/item_feat.csv'

## 特征集合

In [4]:
# 特征名
user_feat_list = []
item_feat_list = []
user_feat_cross = []
numberic_feat_list = []
cat_feat_list = []
embeding_feat_list = []
item_feat_cols = ["item_id"]+['text_vd' + str(i) for i in range(128)]+['img_vd' + str(i) for i in range(128)]
user_feat_cols=["user_id","user_age_level","user_gender","user_city_level"]

### item/user基础特征提取

In [5]:
def user_item_base_feat(base_files, columns=[], isitem=True,  list_feat=[]):
    """
    获取用户或者物品的基础特征
    :param base_files:用户/物品的基础路径
    :param columns: 特征重命名
    :param isitem:  是否是物品 True表示是物品，False表示是用户

    :param list_feat:   需要提取特征的list   [item1,item2]
    :return:
    """
    try:

        if (len(columns) <= 0):
            print("特征列明不可为空")
            return
        print(len(columns))
        feat_columns = []

        if len(list_feat) > 0:
            feat_columns = list_feat
        if base_files:
            dt = pd.read_csv(base_files, header=None,names=columns)
            dt = dt.drop_duplicates()  # 去重
            if isitem == True:
                dt[item_feat_cols[1]] = dt[item_feat_cols[1]].apply(lambda x: x[1:])
                dt[item_feat_cols[128]] = dt[item_feat_cols[128]].apply(lambda x: x[:-1])
                dt[item_feat_cols[129]] = dt[item_feat_cols[129]].apply(lambda x: x[1:])
                dt[item_feat_cols[256]] = dt[item_feat_cols[256]].apply(lambda x: x[:-1])
            if len(feat_columns) > 0:
                dt = dt[feat_columns]
            print("读取item特征结束")
            return dt
        else:
            print("请输入文件路径")
    except Exception as e:
        print(e)


### item提取

In [6]:
item_feat=user_item_base_feat(train_path_item,item_feat_cols)

257
读取item特征结束


In [7]:
item_feat.head()
item_feat.info()

item_feat.shape

,item_id,text_vd0,text_vd1,text_vd2,text_vd3,text_vd4,text_vd5,text_vd6,text_vd7,text_vd8,...,img_vd118,img_vd119,img_vd120,img_vd121,img_vd122,img_vd123,img_vd124,img_vd125,img_vd126,img_vd127
0,42844,4.514945030212402,-2.383720,0.500414,0.407068,-1.995229,0.109078,-0.691775,2.227460,-6.437974,...,-3.374727,-1.506969,-1.820180,-3.024644,0.445263,0.013933,-1.300239,2.759948,2.056171,0.5087034702301025
1,67898,-2.0029051303863525,-0.929881,0.790017,-1.380895,-0.510463,-1.810096,1.363962,0.497401,-4.038903,...,-0.538330,-2.620164,1.277195,0.601015,-0.345312,0.993457,1.351633,2.162675,2.768597,-0.9371970295906067
2,66446,4.221673011779785,-1.497139,1.133570,-2.745607,-4.197045,-0.542392,-1.396256,1.838419,-6.066454,...,-4.582711,-1.056910,-2.568084,-2.038061,2.508719,-0.764789,-0.657116,3.252782,2.687366,0.8443320989608765
3,63651,2.6579699516296387,-0.941863,1.121529,-5.109496,-0.279041,-0.351968,-1.086983,2.703607,-6.494977,...,-0.487683,-1.889119,0.943015,-2.834418,1.633184,2.001801,-2.333152,2.645595,2.280233,-0.6944484114646912
4,46824,3.192194938659668,-1.936676,1.199909,-2.562152,-2.573456,0.575841,-2.358653,1.620844,-4.302936,...,-0.621475,-2.091410,0.501600,-3.083864,-1.060091,2.053600,-2.025008,2.399251,2.562317,0.6941336393356323


<class 'pandas.core.frame.DataFrame'>
Int64Index: 108916 entries, 0 to 108915
Columns: 257 entries, item_id to img_vd127
dtypes: float64(252), int64(1), object(4)
memory usage: 214.4+ MB


(108916, 257)

### 用户特征提取

In [8]:
user_feat=user_item_base_feat(train_path_user,user_feat_cols,isitem=False)

4
读取item特征结束


In [9]:
user_feat.head()
user_feat.info()
user_feat.shape

,user_id,user_age_level,user_gender,user_city_level
0,17,8.0,M,4.0
1,26,7.0,M,2.0
2,35,6.0,F,4.0
3,40,6.0,M,1.0
4,49,6.0,M,1.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6789 entries, 0 to 6788
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          6789 non-null   int64  
 1   user_age_level   6706 non-null   float64
 2   user_gender      6708 non-null   object 
 3   user_city_level  6767 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 265.2+ KB


(6789, 4)

## 读取click特征

In [10]:
def get_train_test_click_data(files, names=['user_id', 'item_id', 'time'], num=1,  list_feat=[]):
    """
    获取train/test中click数据
    :param files: 基础数据文件地址
    :param names: 列名
    :param num: 文件个数
    :param list_feat:  需要获取的特征列名list [item1,item2]
    :return:
    """
    try:
        if (len(names)<=0):
            print("特征列名不可为空")
            return
        all_train = pd.DataFrame(columns=names)
        for p in range(num):
            print(p)
            dt = pd.read_csv(files.format(p), header=None,  names=names)
            all_train = pd.concat([all_train, dt])
            del dt
        all_train = all_train.drop_duplicates()  # 去重
        feat_columns = []
        if len(list_feat) > 0:
            feat_columns = list_feat
        if len(feat_columns) > 0:
            all_train = all_train[feat_columns]
        print("读取click特征结束")
        return all_train
    except Exception as e:
        print(e)

## 获取traink_click特征

In [11]:
train_click_data=get_train_test_click_data(train_path_click,names=['user_id', 'item_id', 'time'],num=7,list_feat=[])

0
1
2
3
4
5
6
读取click特征结束


In [12]:
train_click_data.head()
train_click_data.shape

,user_id,item_id,time
0,4965,18,0.983763
1,20192,34,0.983772
2,30128,91,0.983780
3,29473,189,0.983930
4,10625,225,0.983925


(1139080, 3)

### 用户出现次数

In [13]:
train_click_data["user_id_showup_count"] = train_click_data['user_id'].map(train_click_data.groupby('user_id').size())
train_click_data.head()

,user_id,item_id,time,user_id_showup_count
0,4965,18,0.983763,151
1,20192,34,0.983772,127
2,30128,91,0.983780,65
3,29473,189,0.983930,50
4,10625,225,0.983925,112


In [14]:
train_click_data["user_id_item_unique"] = train_click_data['user_id'].map(train_click_data.groupby('user_id')["item_id"].size())
train_click_data.head()

,user_id,item_id,time,user_id_showup_count,user_id_item_unique
0,4965,18,0.983763,151,151
1,20192,34,0.983772,127,127
2,30128,91,0.983780,65,65
3,29473,189,0.983930,50,50
4,10625,225,0.983925,112,112


### 用户不同点击不同Item个数

In [15]:
train_click_data["user_id_item_unique"] = train_click_data['user_id'].map(train_click_data.groupby('user_id')["item_id"].nunique())
train_click_data.head()

,user_id,item_id,time,user_id_showup_count,user_id_item_unique
0,4965,18,0.983763,151,115
1,20192,34,0.983772,127,100
2,30128,91,0.983780,65,57
3,29473,189,0.983930,50,35
4,10625,225,0.983925,112,77


### 用户对不同item点击的次数

In [16]:
user_item_count = train_click_data.groupby(['user_id', "item_id"]).size().reset_index()
user_item_count.rename(columns={0: "user_id_item_count"}, inplace=True)

In [17]:
user_item_count.head()

,user_id,item_id,user_id_item_count
0,1,2945,1
1,1,4033,1
2,1,4889,1
3,1,11624,1
4,1,13025,1


In [18]:
train_click_data = pd.merge(train_click_data, user_item_count, how='left', on=['user_id', "item_id"])
train_click_data.head()

,user_id,item_id,time,user_id_showup_count,user_id_item_unique,user_id_item_count
0,4965,18,0.983763,151,115,3
1,20192,34,0.983772,127,100,1
2,30128,91,0.983780,65,57,1
3,29473,189,0.983930,50,35,1
4,10625,225,0.983925,112,77,2


### 获取不同时间长度的时间序列下用户点击物品的次数

In [19]:
for i in range(5, 14):
        col = "time_top_{0}".format(i)
        train_click_data[col] = train_click_data['time'].apply(lambda x: str(x)[:i])
#不同时间序列下用户点击的次数
for i in range(5, 14):
    col = "time_top_{0}".format(i)
    test_click_n = train_click_data.groupby(['user_id', col]).size().reset_index()
    col_count = col + "_user_clickcount"
    test_click_n.rename(columns={0: col_count}, inplace=True)
    train_click_data = pd.merge(train_click_data, test_click_n, how='left', on=['user_id', col])

In [20]:
train_click_data.head()

,user_id,item_id,time,user_id_showup_count,user_id_item_unique,user_id_item_count,time_top_5,time_top_6,time_top_7,time_top_8,...,time_top_13,time_top_5_user_clickcount,time_top_6_user_clickcount,time_top_7_user_clickcount,time_top_8_user_clickcount,time_top_9_user_clickcount,time_top_10_user_clickcount,time_top_11_user_clickcount,time_top_12_user_clickcount,time_top_13_user_clickcount
0,4965,18,0.983763,151,115,3,0.983,0.9837,0.98376,0.983763,...,0.98376347602,68,7,2,2,2,1,1,1,1
1,20192,34,0.983772,127,100,1,0.983,0.9837,0.98377,0.983772,...,0.98377237028,35,10,8,2,2,1,1,1,1
2,30128,91,0.983780,65,57,1,0.983,0.9837,0.98378,0.983780,...,0.98378018635,33,22,19,6,3,2,1,1,1
3,29473,189,0.983930,50,35,1,0.983,0.9839,0.98393,0.983930,...,0.98393014840,43,10,2,2,1,1,1,1,1
4,10625,225,0.983925,112,77,2,0.983,0.9839,0.98392,0.983925,...,0.98392539971,71,27,7,6,1,1,1,1,1


In [22]:
train_click_data1=train_click_data[train_click_data["item_id"].isin(item_feat["item_id"])]

In [23]:
train_click_data1.shape

(1012695, 24)

In [24]:
train_click_data1.head()

,user_id,item_id,time,user_id_showup_count,user_id_item_unique,user_id_item_count,time_top_5,time_top_6,time_top_7,time_top_8,...,time_top_13,time_top_5_user_clickcount,time_top_6_user_clickcount,time_top_7_user_clickcount,time_top_8_user_clickcount,time_top_9_user_clickcount,time_top_10_user_clickcount,time_top_11_user_clickcount,time_top_12_user_clickcount,time_top_13_user_clickcount
0,4965,18,0.983763,151,115,3,0.983,0.9837,0.98376,0.983763,...,0.98376347602,68,7,2,2,2,1,1,1,1
2,30128,91,0.983780,65,57,1,0.983,0.9837,0.98378,0.983780,...,0.98378018635,33,22,19,6,3,2,1,1,1
3,29473,189,0.983930,50,35,1,0.983,0.9839,0.98393,0.983930,...,0.98393014840,43,10,2,2,1,1,1,1,1
4,10625,225,0.983925,112,77,2,0.983,0.9839,0.98392,0.983925,...,0.98392539971,71,27,7,6,1,1,1,1,1
5,2426,233,0.983937,69,47,1,0.983,0.9839,0.98393,0.983937,...,0.98393701940,28,18,9,1,1,1,1,1,1


In [25]:
train_click_merge_item1=pd.merge(train_click_data1,item_feat,on="item_id",how="left")

In [26]:
train_click_merge_item1.shape

(1012695, 280)

In [27]:
train_click_merge_item1.head()

,user_id,item_id,time,user_id_showup_count,user_id_item_unique,user_id_item_count,time_top_5,time_top_6,time_top_7,time_top_8,...,img_vd118,img_vd119,img_vd120,img_vd121,img_vd122,img_vd123,img_vd124,img_vd125,img_vd126,img_vd127
0,4965,18,0.983763,151,115,3,0.983,0.9837,0.98376,0.983763,...,0.154401,-1.103010,1.529189,-2.434403,1.647153,1.607877,2.222722,3.103904,2.250426,1.4363162517547607
1,30128,91,0.983780,65,57,1,0.983,0.9837,0.98378,0.983780,...,-0.956756,-1.188611,-3.163749,0.813323,-1.103815,-4.071686,1.281345,2.270864,2.436819,-1.7316410541534424
2,29473,189,0.983930,50,35,1,0.983,0.9839,0.98393,0.983930,...,2.507490,1.911365,1.958169,0.609211,2.857615,2.223901,-0.218719,0.338132,-3.288355,-2.5667481422424316
3,10625,225,0.983925,112,77,2,0.983,0.9839,0.98392,0.983925,...,0.068186,1.279246,-1.896085,-1.728785,-2.019650,2.530539,2.173774,0.050129,2.039675,1.2305697202682495
4,2426,233,0.983937,69,47,1,0.983,0.9839,0.98393,0.983937,...,-0.795265,-1.678250,4.102212,-1.519939,-3.131088,1.953540,3.181193,-0.276408,0.311190,-3.65940523147583


## 获取物品的特征

In [28]:
def get_item_feat(files, names=['user_id', 'item_id', 'time'], num=1,  list_feat=[]):
    """
    获取物品累计特征
    :param files:
    :param names:
    :param num:
    :param list_feat:
    :return:
    """
    #获取基础数据
    item_click_data = get_train_test_click_data(train_path_click, names=['user_id', 'item_id', 'time'], num=7,list_feat=[])
    # 获取不同时间长度的时间序列
    feat_col=names
    for i in range(5, 14):
        col = "time_top_{0}".format(i)
        item_click_data[col] = item_click_data['time'].apply(lambda x: str(x)[:i])
    # 不同时间序列下物品被点击的次数
    for i in range(5, 14):
        col = "time_top_{0}".format(i)
        test_click_n = item_click_data.groupby(['item_id', col]).size().reset_index()
        col_count = col + "_item_clickcount"
        feat_col.append(col_count)
        test_click_n.rename(columns={0: col_count}, inplace=True)
        item_click_data = pd.merge(item_click_data, test_click_n, how='left', on=['item_id', col])
    item_click_data=item_click_data[feat_col]
    return item_click_data

In [29]:
item_click_item=get_item_feat(train_path_click, names=['user_id', 'item_id', 'time'], num=7,  list_feat=[])

0
1
2
3
4
5
6
读取click特征结束


In [30]:
item_click_item.shape

(1139080, 12)

In [31]:
item_click_item.head()

,user_id,item_id,time,time_top_5_item_clickcount,time_top_6_item_clickcount,time_top_7_item_clickcount,time_top_8_item_clickcount,time_top_9_item_clickcount,time_top_10_item_clickcount,time_top_11_item_clickcount,time_top_12_item_clickcount,time_top_13_item_clickcount
0,4965,18,0.983763,9,3,1,1,1,1,1,1,1
1,20192,34,0.983772,22,2,2,1,1,1,1,1,1
2,30128,91,0.983780,19,3,1,1,1,1,1,1,1
3,29473,189,0.983930,4,3,1,1,1,1,1,1,1
4,10625,225,0.983925,8,7,2,2,1,1,1,1,1


In [32]:
train_click_merge_item1=pd.merge(train_click_merge_item1,item_click_item,on=["user_id","item_id"],how="left")

In [33]:
train_click_merge_item1.shape

(1529637, 290)

In [34]:
train_click_merge_item1.head

<bound method NDFrame.head of         user_id item_id    time_x  user_id_showup_count  user_id_item_unique  \
0          4965      18  0.983763                   151                  115   
1          4965      18  0.983763                   151                  115   
2          4965      18  0.983763                   151                  115   
3         30128      91  0.983780                    65                   57   
4         29473     189  0.983930                    50                   35   
...         ...     ...       ...                   ...                  ...   
1529632   17241  117475  0.984068                    73                   50   
1529633   11654  117475  0.984068                    87                   82   
1529634   18127  117475  0.984068                    75                   75   
1529635   12478  117529  0.984087                    94                   76   
1529636     100  117529  0.984227                    62                   51   

         

### 聚类研究 

In [35]:
from sklearn.cluster import KMeans

In [36]:
item_feat

,item_id,text_vd0,text_vd1,text_vd2,text_vd3,text_vd4,text_vd5,text_vd6,text_vd7,text_vd8,...,img_vd118,img_vd119,img_vd120,img_vd121,img_vd122,img_vd123,img_vd124,img_vd125,img_vd126,img_vd127
0,42844,4.514945030212402,-2.383720,0.500414,0.407068,-1.995229,0.109078,-0.691775,2.227460,-6.437974,...,-3.374727,-1.506969,-1.820180,-3.024644,0.445263,0.013933,-1.300239,2.759948,2.056171,0.5087034702301025
1,67898,-2.0029051303863525,-0.929881,0.790017,-1.380895,-0.510463,-1.810096,1.363962,0.497401,-4.038903,...,-0.538330,-2.620164,1.277195,0.601015,-0.345312,0.993457,1.351633,2.162675,2.768597,-0.9371970295906067
2,66446,4.221673011779785,-1.497139,1.133570,-2.745607,-4.197045,-0.542392,-1.396256,1.838419,-6.066454,...,-4.582711,-1.056910,-2.568084,-2.038061,2.508719,-0.764789,-0.657116,3.252782,2.687366,0.8443320989608765
3,63651,2.6579699516296387,-0.941863,1.121529,-5.109496,-0.279041,-0.351968,-1.086983,2.703607,-6.494977,...,-0.487683,-1.889119,0.943015,-2.834418,1.633184,2.001801,-2.333152,2.645595,2.280233,-0.6944484114646912
4,46824,3.192194938659668,-1.936676,1.199909,-2.562152,-2.573456,0.575841,-2.358653,1.620844,-4.302936,...,-0.621475,-2.091410,0.501600,-3.083864,-1.060091,2.053600,-2.025008,2.399251,2.562317,0.6941336393356323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108911,79253,2.1436519622802734,-1.591184,-0.283598,-2.186552,-1.505779,0.876601,1.302617,-1.762244,-2.847601,...,-1.352005,-3.588055,2.803221,-2.160821,-1.153775,0.290783,1.518086,3.690181,5.391709,0.3289714455604553
108912,109138,0.8901849985122681,0.042669,2.842594,-4.322702,-1.107593,-0.033230,2.835225,-1.383299,-4.998042,...,0.416093,-0.917160,-0.028134,2.624365,1.574156,-0.014772,2.250575,2.005705,0.309760,0.9139738082885742
108913,62184,1.4589283466339111,-0.638677,-0.957509,-2.936515,-0.897658,-0.992379,0.574731,0.027216,-3.514133,...,1.468603,-2.667665,-0.988566,1.012626,1.046013,1.670171,0.804473,2.821836,3.285690,2.2152724266052246
108914,42748,3.3835203647613525,-1.669863,1.264212,-2.128029,-2.129893,2.562061,-1.778324,-0.861360,-5.312111,...,-1.577673,-1.983800,-0.830833,-2.517840,-0.387712,0.237070,-2.524445,1.908586,1.887787,1.0216277837753296


In [37]:
loan = np.array(item_feat[['text_vd' + str(i) for i in range(128)]])

In [38]:
clf=KMeans(n_clusters=100)

In [39]:
clf=clf.fit(loan)

In [40]:
clf.labels_

array([25,  5,  7, ..., 14, 62, 14])

In [42]:
item_feat['text_cat']=clf.labels_

In [43]:
item_feat.head()

,item_id,text_vd0,text_vd1,text_vd2,text_vd3,text_vd4,text_vd5,text_vd6,text_vd7,text_vd8,...,img_vd119,img_vd120,img_vd121,img_vd122,img_vd123,img_vd124,img_vd125,img_vd126,img_vd127,text_cat
0,42844,4.514945030212402,-2.383720,0.500414,0.407068,-1.995229,0.109078,-0.691775,2.227460,-6.437974,...,-1.506969,-1.820180,-3.024644,0.445263,0.013933,-1.300239,2.759948,2.056171,0.5087034702301025,25
1,67898,-2.0029051303863525,-0.929881,0.790017,-1.380895,-0.510463,-1.810096,1.363962,0.497401,-4.038903,...,-2.620164,1.277195,0.601015,-0.345312,0.993457,1.351633,2.162675,2.768597,-0.9371970295906067,5
2,66446,4.221673011779785,-1.497139,1.133570,-2.745607,-4.197045,-0.542392,-1.396256,1.838419,-6.066454,...,-1.056910,-2.568084,-2.038061,2.508719,-0.764789,-0.657116,3.252782,2.687366,0.8443320989608765,7
3,63651,2.6579699516296387,-0.941863,1.121529,-5.109496,-0.279041,-0.351968,-1.086983,2.703607,-6.494977,...,-1.889119,0.943015,-2.834418,1.633184,2.001801,-2.333152,2.645595,2.280233,-0.6944484114646912,41
4,46824,3.192194938659668,-1.936676,1.199909,-2.562152,-2.573456,0.575841,-2.358653,1.620844,-4.302936,...,-2.091410,0.501600,-3.083864,-1.060091,2.053600,-2.025008,2.399251,2.562317,0.6941336393356323,7


In [50]:
from time import time
def bench_k_means(estimator, name, data):
    t0 = time()
    result=estimator.fit(data)
    return result

In [46]:
item_feat_col=user_item_base_feat(train_path_item,item_feat_cols)

257
读取item特征结束


In [51]:

feat_kmeans=bench_k_means(KMeans(init='k-means++', n_clusters=100, n_init=10),
              name="k-means++", data=item_feat_col)
feat_kmeans.cluster_centers_
feat_kmeans.labels_

array([[ 4.83708134e+04,  1.70870150e+00, -3.65532524e-01, ...,
         2.34408736e+00,  1.79467147e+00, -1.89949003e-01],
       [ 8.90506744e+04,  1.38777112e+00, -2.84511577e-01, ...,
         2.40383498e+00,  1.78607541e+00, -3.50545412e-01],
       [ 1.82564430e+04,  1.72653832e+00, -1.96590061e-01, ...,
         2.28581323e+00,  1.79308953e+00, -3.52418644e-01],
       ...,
       [ 1.59462578e+04,  1.69408367e+00, -1.58185263e-01, ...,
         2.30717799e+00,  1.85668376e+00, -4.13285700e-01],
       [ 4.03972290e+04,  1.75260205e+00, -3.52004079e-01, ...,
         2.32603727e+00,  1.90647462e+00, -2.96229076e-01],
       [ 4.38145504e+04,  1.65375860e+00, -4.03708550e-01, ...,
         2.32798356e+00,  1.92785500e+00, -1.73620294e-01]])

array([49, 23, 78, ...,  9, 49, 66])

In [52]:
feat_kmeans.inertia_

12611256550.510286

In [53]:
feat_kmeans.n_iter_ 

61

In [60]:
import time
from sklearn.metrics import silhouette_score,calinski_harabaz_score
clusters = []

In [61]:
for i in range(10,160,10):
    print(i)
    clusters.append(i)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


In [62]:
clusters

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]

In [ ]:
silhouette_scores = []
#轮廓系数silhouette_scores
print('start time: ',time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
for k in clusters:
    y_pred = KMeans(n_clusters = k, verbose = 0, n_jobs = -1, random_state=1).fit_predict(item_feat_col)
    score = silhouette_score(item_feat_col, y_pred)
    silhouette_scores.append(score)
print('finish time: ',time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
plt.plot(clusters, silhouette_scores, '*-')
plt.xlabel('k')
plt.ylabel('silhouette_score')
plt.show()


start time:  2020-05-20 23:29:55
